In [1]:
import os
import h5py
import numpy as np
from torchdeepretina.datas import loadexpt, _loadexpt_h5

In [2]:
data_path = os.path.join('/home/salamander/experiments/data/')

In [5]:
data = h5py.File(data_path, mode='r')

In [9]:
data.keys()

<KeysViewHDF5 ['spikes', 'test', 'train']>

In [11]:
data['spikes'].keys()

<KeysViewHDF5 ['cell01', 'cell02', 'cell03', 'cell04', 'cell05', 'cell06', 'cell07', 'cell08', 'cell09']>

In [14]:
data['train'].keys()

<KeysViewHDF5 ['response', 'stimulus', 'time']>

In [18]:
np.asarray(data['train']['stimulus']).shape

(359802, 50, 50)

In [20]:
data['train']['response'].keys()

<KeysViewHDF5 ['binned', 'firing_rate_10ms', 'firing_rate_20ms', 'firing_rate_5ms']>

In [21]:
np.asarray(data['train']['response']['binned']).shape

(9, 359802)

In [29]:
np.asarray(data['train']['response']['firing_rate_10ms']).shape

(9, 359802)

In [47]:
print(np.asarray(data['train']['response']['firing_rate_10ms'])[0,6])

0.47261764030129316


In [48]:
print(np.asarray(data['train']['response']['binned'])[0,6])

0.0


In [3]:
from xhding.models import KineticsModel
import torch
from collections import deque

In [4]:
model = KineticsModel()


In [5]:
def get_hs(model, batch_size, device):
    hs = []
    hs.append(torch.zeros(batch_size, *model.h_shapes[0]).to(device))
    hs[0][:,0] = 1
    hs.append(deque([],maxlen=model.seq_len))
    for i in range(model.seq_len):
        hs[1].append(torch.zeros(batch_size, *model.h_shapes[1]).to(device))
    return hs

In [4]:
hs = get_hs(model, 5)

In [7]:
batch_size = 5
x = torch.ones(batch_size, 40, 50, 50)

In [10]:
out, hs = model(x, hs)

In [11]:
out.shape

torch.Size([5, 5])

In [4]:
from torch.utils.data.sampler import Sampler
from torch.utils.data.sampler import SequentialSampler
class BatchRnnSampler(Sampler):
    
    def __init__(self, sampler, batch_size):
        self.sampler = sampler
        self.batch_size = batch_size

    def __iter__(self):
        for batch_idx in range(self.__len__()):
            batch = [batch_idx + n * self.__len__() for n in range(self.batch_size)]
            yield batch

    def __len__(self):
        return len(self.sampler) // self.batch_size

In [18]:
list(BatchRnnSampler(SequentialSampler(range(10)), batch_size=3))

[[0, 3, 6], [1, 4, 7], [2, 5, 8]]

In [23]:
data = loadexpt('15-10-07', [0,1,2,3,4], 'naturalscene', 'train', 40, 0, data_path=data_path)

In [24]:
data.X.shape

(359762, 40, 50, 50)

In [26]:
data.y.shape

(359762, 5)

In [6]:
from xhding.data import BatchRnnSampler, TrainDataset, ValidationDataset
dataset = ValidationDataset(data_path)

In [7]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SequentialSampler
batch_sampler = BatchRnnSampler(SequentialSampler(range(len(dataset))), batch_size=512)
data = DataLoader(dataset=dataset)

In [8]:
for i,(x,y) in enumerate(data):
    if i == 0:
        inpt = x
        trgt = y
        break

In [9]:
inpt.shape

torch.Size([512, 40, 50, 50])

In [ ]:
dataset

In [8]:
from scipy.stats import pearsonr
device = torch.device('cuda:1')
hs = get_hs(model, 1, device)
model = model.to(device)
model.eval()
with torch.no_grad():
    pearsons = []
    val_pred = []
    val_targ = []
    for x,y in data:
        x = x.to(device)
        out, hs = model(x, hs)
        val_pred.append(out.detach().cpu().numpy().squeeze())
        val_targ.append(y.detach().numpy().squeeze())
    val_pred = np.stack(val_pred, axis=0)
    val_targ = np.stack(val_targ, axis=0)
    for cell in range(5):
        pearsons.append(pearsonr(val_pred[:,cell],val_targ[:,cell])[0])
    print(np.array(pearsons).mean())

-0.0035025242913605073


In [10]:
val_targ.shape

(30000, 5)